<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [2]:
pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=ad42237c7fa2989684b0ce46a5b9d6aa3138dc811b0466e0095ee37a7d871502
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import preprocess_input
import os
import zipfile
import wget

In [4]:
# Download the data
url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip'
filename = 'concrete_data_week4.zip'
wget.download(url, filename)

# Extract the data
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('concrete_data_week4')

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [10]:
# Set paths for train, validation, and test directories
train_dir = '/content/concrete_data_week4/concrete_data_week4/train'
valid_dir = '/content/concrete_data_week4/concrete_data_week4/valid'
test_dir = '/content/concrete_data_week4/concrete_data_week4/test'

In [11]:
# Batch size
batch_size = 100

# Image data generators with data augmentation for training
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [12]:
# Image data generators for validation
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [13]:
# Flow from directory for training
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 30001 images belonging to 2 classes.


In [14]:
# Flow from directory for validation
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 9501 images belonging to 2 classes.


In [15]:
# Load the VGG16 model
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base
vgg16_base.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


In [25]:
# Create the model
model = Sequential()
model.add(vgg16_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Assuming 10 classes in the dataset

In [26]:
# Print the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 25088)             0         
                                                                 
 dense_4 (Dense)             (None, 256)               6422784   
                                                                 
 dense_5 (Dense)             (None, 1)                 257       
                                                                 
Total params: 21137729 (80.63 MB)
Trainable params: 6423041 (24.50 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [27]:
# Compile the model
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [29]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=3
)

Epoch 1/3
300/300 [==============================] - 513s 2s/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/3
300/300 [==============================] - 470s 2s/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/3
300/300 [==============================] - 510s 2s/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000


<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model.
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [30]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import load_model

# Load the ResNet50 model
resnet50_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base
resnet50_base.trainable = False

# Create the model
resnet_model = Sequential()
resnet_model.add(resnet50_base)
resnet_model.add(Flatten())
resnet_model.add(Dense(256, activation='relu'))
resnet_model.add(Dense(1, activation='sigmoid'))  # Assuming 10 classes in the dataset

# Compile the model
resnet_model.compile(optimizer=Adam(),
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

# Train the ResNet50 model
resnet_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=3
)

# Save the model
resnet_model.save('resnet_model.h5')


94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/3
300/300 [==============================] - 515s 1s/step - loss: 1.3511 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/3
300/300 [==============================] - 443s 1s/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/3
300/300 [==============================] - 446s 1s/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [31]:
# Image data generator for the test set
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Flow from directory for test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 500 images belonging to 2 classes.


In [32]:
# Evaluate the VGG16 model
vgg16_performance = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'VGG16 Model Test Loss: {vgg16_performance[0]}')
print(f'VGG16 Model Test Accuracy: {vgg16_performance[1]}')


5/5 [==============================] - 2s 435ms/step - loss: 0.6931 - accuracy: 0.5000
VGG16 Model Test Loss: 0.6931472420692444
VGG16 Model Test Accuracy: 0.5


In [33]:
# Load the saved ResNet50 model
saved_resnet_model = load_model('resnet_model.h5')

# Evaluate the ResNet50 model
resnet_performance = saved_resnet_model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'ResNet50 Model Test Loss: {resnet_performance[0]}')
print(f'ResNet50 Model Test Accuracy: {resnet_performance[1]}')

5/5 [==============================] - 3s 333ms/step - loss: 0.6931 - accuracy: 0.5000
ResNet50 Model Test Loss: 0.6931472420692444
ResNet50 Model Test Accuracy: 0.5


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li>
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [34]:
# Predict the classes for the test data using the VGG16 model
vgg16_predictions = model.predict(test_generator, steps=test_generator.samples // batch_size, verbose=1)

# Predict the classes for the test data using the ResNet50 model
resnet_predictions = saved_resnet_model.predict(test_generator, steps=test_generator.samples // batch_size, verbose=1)

5/5 [==============================] - 3s 350ms/step


In [35]:
# Get class indices
class_indices = {v: k for k, v in test_generator.class_indices.items()}
print(class_indices)  # To see the mapping of class indices

# Decode the first five predictions for VGG16
vgg16_class_labels = ['Positive' if pred == 1 else 'Negative' for pred in vgg16_predictions.argmax(axis=-1)[:5]]

# Decode the first five predictions for ResNet50
resnet_class_labels = ['Positive' if pred == 1 else 'Negative' for pred in resnet_predictions.argmax(axis=-1)[:5]]

{0: 'negative', 1: 'positive'}


In [38]:
# Print the class predictions for the first five images in the test set for VGG16
print("VGG16 Model Predictions:")
print("<ul style='list-style-type:none'>")
for label in vgg16_class_labels:
    print({label})
print("</ul>")

# Print the class predictions for the first five images in the test set for ResNet50
print("ResNet50 Model Predictions:")
print("<ul style='list-style-type:none'>")
for label in resnet_class_labels:
    print({label})
print("</ul>")

VGG16 Model Predictions:
<ul style='list-style-type:none'>
{'Negative'}
{'Negative'}
{'Negative'}
{'Negative'}
{'Negative'}
</ul>
ResNet50 Model Predictions:
<ul style='list-style-type:none'>
{'Negative'}
{'Negative'}
{'Negative'}
{'Negative'}
{'Negative'}
</ul>


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).